# Descripción

Contamos con unos datasets que corresponden a un **listado de inspecciones de sanidad en locales** (Restaurantes, supermercados, etc), junto con su respectivo riesgo para la salud. Contamos con otro dataset que nos muestra una **descripción de dicho riesgo**.

**El objetivo es cargar esos datasets bajo unas especificaciones concretas y manipularlos acorde a las instrucciones de cada ejercicio.**

Todas las operaciones necesarias están descritas en los ejercicios, aunque se valorará tareas extras por propia iniciativa del alumno. También se valorará el uso del API de DataFrame

**La entrega será este fichero "ipynb" (el nombre del fichero será vuestro nombre), junto con la imagen del plan de ejecución del ejercicio 4**, comprimido en un fichero zip con el nombre y apellidos del alumno

# Descargar Datasets

In [0]:
%sh 
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/food_inspections_lite.csv'
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/risk_description.csv'

In [0]:
dbutils.fs.cp('file:/databricks/driver/food_inspections_lite.csv','dbfs:/dataset/food_inspections_lite.csv')
dbutils.fs.cp('file:/databricks/driver/risk_description.csv','dbfs:/dataset/risk_description.csv')

In [0]:
dbutils.fs.ls('/dataset/')

# Ejercicio 1
---

1. **Crea dos dataframes, uno a partir del fichero `food_inspections_lite.csv` y otro a partir de `risk_description.csv`**
2. **Convierte esos dos dataframes a tablas delta**


In [0]:
# Eliminación de las tablas para evitar conflictos con posibles datos antiguos

# Definir la ruta de las tablas Delta
delta_food_inspections_path = "dbfs:/delta/food_inspections"
delta_risk_description_path = "dbfs:/delta/risk_description"

# Eliminar las tablas Delta si ya existen
spark.sql(f"DROP TABLE IF EXISTS delta.`{delta_food_inspections_path}`")
spark.sql(f"DROP TABLE IF EXISTS delta.`{delta_risk_description_path}`")

# Crear las tablas Delta nuevamente
spark.sql(f"CREATE TABLE IF NOT EXISTS food_inspections USING DELTA LOCATION '{delta_food_inspections_path}'")
spark.sql(f"CREATE TABLE IF NOT EXISTS risk_description USING DELTA LOCATION '{delta_risk_description_path}'")


In [0]:
# Imprimir el contenido de los ficheros food_inspections_lite.csv y risk_description.csv para comprobar los datos

import pandas as pd

# Definir las rutas de los archivos CSV
food_inspections_path = "file:/databricks/driver/food_inspections_lite.csv"
risk_description_path = "file:/databricks/driver/risk_description.csv"

# Leer los ficheros CSV
df_food_inspections = pd.read_csv(food_inspections_path)
df_risk_description = pd.read_csv(risk_description_path)

# Imprimir primeras filas de los ficheros
print(df_food_inspections.head())
print(df_risk_description.head())

In [0]:
# Rutas de los ficheros CSV almacenados en DBFS (Databricks File System)
food_inspections_path = "dbfs:/dataset/food_inspections_lite.csv"
risk_description_path = "dbfs:/dataset/risk_description.csv"

# Crear el DataFrame a partir del fichero food_inspections_lite.csv
df_food_inspections = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(food_inspections_path)

# Crear el DataFrame a partir del fichero risk_description.csv
df_risk_description = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(risk_description_path)

In [0]:
# Imprimir Dataframes
df_food_inspections.show()
df_risk_description.show(truncate=False)

In [0]:
# Imprimir esquemas de los DataFrames
df_food_inspections.printSchema()
df_risk_description.printSchema()

In [0]:
# Limpieza del nombre de las columnas por seguridad, compatibilidad y estabilidad.

from pyspark.sql.functions import col

def clean_column_names(df):
    for col_name in df.columns:
        # Convertir a minúsculas
        cleaned_name = col_name.lower()
        # Reemplazar espacios y caracteres especiales
        cleaned_name = cleaned_name.strip().replace(' ', '_').replace(';', '').replace('{', '').replace('}', '') \
                                            .replace('(', '').replace(')', '').replace('\n', '').replace('\t', '') \
                                            .replace('=', '').replace('#', '')
        # Reglas específicas para columnas conocidas
        if cleaned_name == 'license_':
            cleaned_name = 'license'
        df = df.withColumnRenamed(col_name, cleaned_name)
    return df

df_food_inspections_cleaned = clean_column_names(df_food_inspections)
df_risk_description_cleaned = clean_column_names(df_risk_description)

print("Columnas originales df_food_inspections:")
print(df_food_inspections.columns)

print("Columnas df_food_inspections después de la limpieza:")
print(df_food_inspections_cleaned.columns)

print("Columnas originales df_risk_description:")
print(df_risk_description.columns)

print("Columnas df_risk_description después de la limpieza:")
print(df_risk_description_cleaned.columns)


In [0]:
# Paso 2º: Convertir y escribir los DataFrames (limpios) en tablas Delta:

# Especificar las rutas de salida para las tablas Delta
delta_food_inspections_path = "dbfs:/delta/food_inspections"
delta_risk_description_path = "dbfs:/delta/risk_description"

# Convertir y escribir el DataFrame food_inspections limpio como tabla Delta
df_food_inspections_cleaned.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(delta_food_inspections_path)

# Convertir y escribir el DataFrame risk_description limpio como tabla Delta
df_risk_description_cleaned.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(delta_risk_description_path)

# Imprimir esquemas
df_food_inspections_cleaned.printSchema()
df_risk_description_cleaned.printSchema()

In [0]:
# Registrar tablas Delta en Spark SQL (para realizar consultas SQL sin especificar la ruta en cada operación (entre otros motivos))
spark.sql("CREATE TABLE IF NOT EXISTS food_inspections USING DELTA LOCATION 'dbfs:/delta/food_inspections'")
spark.sql("CREATE TABLE IF NOT EXISTS risk_description USING DELTA LOCATION 'dbfs:/delta/risk_description'")

In [0]:
# Consultar la tabla food_inspections
spark.sql("SELECT * FROM food_inspections LIMIT 10").show()

In [0]:
# Consultar la tabla risk_description
spark.sql("SELECT * FROM risk_description").show(truncate=False)

# Ejercicio 2
**Obtén el número de inspecciones distintas con Riesgo alto `Risk 1 (High)`**

---



Opción 1 resolución ejercicio haciendo un filtrado por con 'Risk 1 (High)' en el dataset 'food_inspections' columna 'Risk' y contar las inspecciones distintas según su ID

In [0]:
# Leer el DataFrame desde Delta Lake
df_food_inspections = spark.read.format("delta").load("dbfs:/delta/food_inspections")

# Filtrar las inspecciones con riesgo alto (Risk 1 (High))
df_high_risk_inspections = df_food_inspections.filter(col('risk') == 'Risk 1 (High)')

# Inspecciones distintas basadas en inspection_id
distinct_high_risk_inspections = df_high_risk_inspections.select('inspection_id').distinct().count()

# Número total de filas en el DataFrame original
total_rows = df_food_inspections.count()
print(f"Filas totales en 'food inspections': {total_rows}")

# Mostrar el número de inspecciones distintas con riesgo alto
print(f"Número de inspecciones distintas con 'Risk 1 (High)': {distinct_high_risk_inspections}")

Opción 2 (más rápida) sin comprobar si las inspecciones son distintas (entendiendo que si el dataset tiene un ID único por fila, todas serán distintas).
El resultado es el mismo.

In [0]:
# Filtrar las filas con 'Risk 1 (High)'
df_risk_high = df_food_inspections.filter(df_food_inspections['Risk'] == 'Risk 1 (High)')

# Número total de filas en el DataFrame original
total_rows = df_food_inspections.count()
print(f"Filas totales en 'food inspections': {total_rows}")

# Contar el número de filas filtradas
count_risk_high = df_risk_high.count()
print(f"Número de filas con 'Risk 1 (High)': {count_risk_high}")

Nota del ejercicio: Al comprobar los nombres únicos en la columna 'Risk' de 'food_inspections' para identificar cuantos restaurantes tienen 'Risk 1 (High)' por si hubiese datos distintos a los indicados en 'df_risk_description', se comprueba que SI LOS HAY (a parte de los riesgos 1, 2 y 3 hay valores 'null' y 'All').

In [0]:
# Valores únicos de la columna 'Risk' en df_food_inspections
df_food_inspections.select("risk").distinct().show()

Se comprueba el nº de filas que contienen 'null' y 'all' en la columna 'Risk' 

In [0]:
# Filtrar y mostrar las filas con valores 'null' en la columna 'Risk'
df_null_risk = df_food_inspections.filter(df_food_inspections["risk"].isNull())

# Número total de filas con valores 'null'
total_null_rows = df_null_risk.count()
df_null_risk.show()
print(f"Filas totales con valor 'null' en columna 'risk': {total_null_rows}")

In [0]:
# Filtrar y mostrar las filas con valor 'All' en la columna 'Risk'
df_all_risk = df_food_inspections.filter(df_food_inspections["risk"] == "All")

# Número total de filas con valores 'all'
total_all_rows = df_all_risk.count()
df_all_risk.show()
print(f"Filas totales con valor 'all' en columna 'risk': {total_all_rows}")

El resultado obtenido es:
- Nº de filas con valores 'null': 5
- Nº de filas con valores 'All': 9

El nº de filas no supone un cambio significativo al suponer un 0.05% y 0.09% respectivamente, pero se procede a la limpieza del dataframe para tener mayor consistencia.

In [0]:
# Limpieza de la columna 'Risk':
# 1. Reemplazar los valores All por Risk 1 (High)
# 2. Eliminar las filas con valores null

from pyspark.sql.functions import col, when

# Cargar la tabla Delta de food inspections
df_food_inspections = spark.read.format("delta").load("dbfs:/delta/food_inspections")

# Reemplazar los valores 'All' por 'Risk 1 (High)'
df_food_inspections_cleaned = df_food_inspections.withColumn(
    "risk", when(col("risk") == "All", "Risk 1 (High)").otherwise(col("risk")))

# Eliminar las filas con valores null en la columna 'risk'
df_food_inspections_cleaned = df_food_inspections_cleaned.filter(col("risk").isNotNull())

# Mostrar los valores únicos de la columna 'risk' después de la limpieza
print("Valores únicos en la columna 'risk' después de la limpieza:")
df_food_inspections_cleaned.select("risk").distinct().show()

# Contar las inspecciones con 'Risk 1 (High)'
df_risk_high = df_food_inspections_cleaned.filter(col("risk") == "Risk 1 (High)")
num_risk_all = df_risk_high.count()

print(f"Número de inspecciones distintas con 'Risk 1 (High)' añadiendo los 'All': {num_risk_all}")


# Ejercicio 3
**A partir de los dataframes cargados anteriormente, obtén una tabla con las siguientes columnas:<br>**
1. `DBA Name`
2. `Facility Type`
3. `Risk`
4. `Risk description`

---

In [0]:
# Realizar join utilizando expresiones para alinear los valores
df_joined = df_food_inspections_cleaned.join(df_risk_description_cleaned, df_food_inspections_cleaned['risk'].contains('Risk'), how='inner')

# Seleccionar las columnas necesarias
df_result = df_joined.select(
  col('dba_name').alias('DBA Name'),
  col('facility_type').alias('Facility Type'),
  col('risk').alias('Risk'),
  col('description').alias('Risk description')
  )

# Mostrar el resultado
df_result.show(truncate=False)


# Ejercicio 4
**Accede a la Spark UI para ver el plan de ejecución del ejercicio anterior (ejercicio 3). Describe cada una de las piezas/cajas que componen el plan de ejecución (Una descripción breve de una línea por caja será suficiente).**<br><br>**Recordad hacer un pantallazo del plan de ejecución analizado y enviadlo junto con este notebook**

---



**Descripción de cada caja en el plan de ejecución del ejercicio 3 en donde se realizó un join entre dos dataframes (df_food_inspections_cleaned y df_risk_description_cleaned)**

**Stage 211 (skipped)**

- Scan csv: Spark escanea y lee los datos desde un archivo CSV.

- WholeStageCodegen: Optimizador que ayuda a reducir la sobrecarga y mejorar la eficiencia en el procesamiento de los datos leídos del CSV.

- Exchange: Redistribuye los datos ente los nodos para preparar la operación de join, generando una etapa de intercambio.

**Stage 212**

- Scan csv: Similar a la caja en el Stage 211, esta operación escanea los datos desde la fuente CSV.

- ShuffleQueryStage: Recibe datos redistribuidos (shuffle) del Exchange en el Stage 211 para la operación de join.

- WholeStageCodegen: Al igual que en el Stage 211, esta caja optimiza la ejecución de los datos combinados y finaliza la ejecución del join.

# Ejercicio 5
**1. Para cada local (columna `DBA Name`) y su resultado (columna `Results`), obtén el número de inspecciones que ha tenido**<br><br>
**2. Obtén los dos locales (`DBA Name`) que más inspecciones han tenido por cada uno de los resultados**<br><br>
**3. Guarda los resultados del punto 2 en una nueva tabla Delta llamada `inspections_results`**

---

In [0]:
# Paso 1: Contar el número de inspecciones por local y resultado

from pyspark.sql.functions import count
# Contar el número de inspecciones por local y resultado
df_count_inspections = df_food_inspections_cleaned.groupBy("dba_name", "results") \
    .agg(count("*").alias("total_inspections")) \
    .orderBy(col("total_inspections").desc())

# Mostrar para verificar
df_count_inspections.show(50, truncate=False)


In [0]:
# Número de inspecciones totales agrupadas por restaurantes
df_count_inspections = df_food_inspections_cleaned.groupBy("dba_name").agg(count("*").alias("total_inspections"))
df_inspection_counts_sorted = df_count_inspections.orderBy(col("total_inspections").desc())
df_inspection_counts_sorted.show(truncate=False)

In [0]:
from pyspark.sql.functions import col, row_number, count
from pyspark.sql.window import Window

# Paso 2: Obtener los dos locales (DBA Name) que más inspecciones han tenido por cada uno de los resultados

# Contar el nº de inspecciones por 'dba_name' y 'results'
df_count_inspections = df_food_inspections.groupBy('dba_name', 'results').agg(count('*').alias('total_inspections'))

# Definir una ventana para particionar por 'results' y ordenar por 'total_inspections' en orden descendente
window_spec = Window.partitionBy("results").orderBy(col("total_inspections").desc())

# Agregar una columna con el número de fila dentro de cada partición
df_ranked = df_count_inspections.withColumn("rank", row_number().over(window_spec))

# Filtrar para obtener solo las dos primeras filas en cada partición
df_top2 = df_ranked.filter(col("rank") <= 2).drop("rank")

# Mostrar el resultado
df_top2.show(truncate=False)

In [0]:
# Paso 3: Guardar los resultados en una nueva tabla Delta

# Guardar los resultados en una nueva tabla Delta llamada inspections_results
df_top2.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("dbfs:/delta/inspections_results")


In [0]:
# Verificar el esquema del DataFrame df_top2
df_top2.printSchema()

# Verificar el esquema de la tabla Delta existente
spark.read.format("delta").load("dbfs:/delta/inspections_results").printSchema()

# Ejercicio 6
1. **Actualiza la tabla delta del ejercicio anterior `inspections_results`, especificando `DBA_Name = error`**<br>
2. **Restaura la tabla a su estado original**

---



In [0]:
# Paso 1: Actualizar la tabla Delta inspections_results

from pyspark.sql.functions import lit

# Leer la tabla Delta inspections_results
df_inspections_results = spark.read.format("delta").load("dbfs:/delta/inspections_results")

# Mostrar algunos registros antes de la actualización
print("Registros antes de la actualización:")
df_inspections_results.show()

# Actualizar la columna DBA_Name con el valor 'error'
df_updated = df_inspections_results.withColumn("dba_name", lit("error"))

# Guardar los cambios en la misma tabla Delta 'inspections_results'
df_updated.write.format("delta").mode("overwrite").save("dbfs:/delta/inspections_results")

# Mostrar registros después de la actualización
print("Registros después de la actualización:")
df_updated.show()


In [0]:
# Paso 2: Restaura la tabla a su estado original

from delta.tables import DeltaTable

# Cargar la tabla Delta
delta_table_path = "dbfs:/delta/inspections_results"
delta_table = DeltaTable.forPath(spark, delta_table_path)

# Mostrar el historial de versiones de la tabla
history_df = delta_table.history()
history_df.display(truncate=False)


In [0]:
# Paso 2º (continuación): Restaurar la tabla a una versión anterior (0), la versión original
previous_version = 0

# Restaurar la tabla a su estado original (versión 0)
spark.sql(f"RESTORE TABLE delta.`{delta_table_path}` TO VERSION AS OF {previous_version}")

# Verificar que la restauración fue exitosa mostrando algunos registros
df_restored = spark.read.format("delta").load(delta_table_path)
df_restored.show()

# Ejercicio 7

**Crea una aplicación con Structured Streaming que lea los datos del topic de Kafka `inspections`. La url del servidor Kafka es `35.237.99.179:9094`:**

**Los datos procedentes de este topic son exactamente los mismos que estamos analizando durante todo este notebook, `Food Inspections`, así que el esquema es el mismo**

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import from_json, col

In [0]:
# Lectura desde Kafka (origen) para verificación de datos

# Configuración servidor y topic de Kafka
kafka_server = "35.237.99.179:9094"
kafka_topic = "inspections"

df_kafka_origin = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

# Mostrar registros
df_kafka_origin.selectExpr("CAST(value AS STRING)").show(truncate=False)


In [0]:
# Configuración de Streaming desde Kafka
df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("subscribe", kafka_topic) \
    .load()

# Definir el esquema para los datos de las inspecciones
schema = StructType([
    StructField("Inspection ID", StringType(), True),
    StructField("DBA Name", StringType(), True),
    StructField("AKA Name", StringType(), True),
    StructField("License #", StringType(), True),
    StructField("Facility Type", StringType(), True),
    StructField("Risk", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Zip", StringType(), True),
    StructField("Inspection Date", StringType(), True),
    StructField("Inspection Type", StringType(), True),
    StructField("Results", StringType(), True),
    StructField("Violations", StringType(), True),
    StructField("Latitude", StringType(), True),
    StructField("Longitude", StringType(), True),
    StructField("Location", StringType(), True)
])

df_kafka.printSchema()

In [0]:
# Convertir los datos desde Kafka usando el esquema definido
dataset = df_kafka.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

In [0]:
# Opción 1 - Mostrar dataset con 'display'
# Permite ver los datos en tiempo real de forma sencilla pero aporta menos versatilidad y control
# Sólo válido en entornos de pruebas, NO USAR EN ENTORNOS DE PRODUCCIÓN

dataset.display()

In [0]:
# Opción 2 - Mostrar dataset con 'writeStream'
# Almacena los resultados en la tabla temporal 'inspections_topic' permitiendo mayor versatilidad y control
# Para ver los resultados, esperar a que estén almacenados y ejecutar la consulta SQL

# Escribir los datos en memoria
dataset.writeStream \
    .outputMode("append") \
    .format("memory") \
    .option("truncate", "false") \
    .queryName("inspections_topic") \
    .start()

In [0]:
%sql
-- Consultar dataset
SELECT * FROM inspections_topic

# Ejercicio 8
**En base a la fuente de datos del ejercicio anterior, obtén cada 5 segundos el número de inspecciones por `Facility Type`**

In [0]:
# Opción 1 - Mostrar dataset con 'display'
# Permite ver los datos en tiempo real de forma sencilla pero aporta menos versatilidad y control
# Sólo válido en entornos de pruebas, NO USAR EN ENTORNOS DE PRODUCCIÓN

from pyspark.sql.functions import col, window
(
dataset
  .groupBy(window(col("timestamp"), "5 seconds"), col("Facility Type"))
  .count()
  .display()
)

In [0]:
# Opción 2 - Mostrar dataset con 'writeStream'
# Almacena los resultados en la tabla temporal 'inspections_update_topic' permitiendo mayor versatilidad y control
# Para ver los resultados, esperar a que estén almacenados y ejecutar la consulta SQL

from pyspark.sql.functions import col, window
(
dataset
  .groupBy(window(col("timestamp"), "5 seconds"), col("Facility Type"))
  .count()
  .writeStream \
  .outputMode("update") \
  .format("memory") \
  .option("truncate", "false") \
  .queryName("inspections_update_topic") \
  .start()
)

In [0]:
%sql
-- Consultar dataset
select * from inspections_update_topic

# Ejercicio 9
**En base a la fuente de datos del ejercicio 7, obtén cada 5 segundos el número de inspecciones por `Results` de los últimos 30 segundos**

In [0]:
# Opción 1 - Mostrar dataset con 'display'
# Permite ver los datos en tiempo real de forma sencilla pero aporta menos versatilidad y control
# Sólo válido en entornos de pruebas, NO USAR EN ENTORNOS DE PRODUCCIÓN

from pyspark.sql.functions import col, window
(
dataset
  .groupBy(window(col("timestamp"), "30 seconds", "5 seconds"), col("results"))
  .count()
  .display()
)

In [0]:
# Opción  2 - Usando 'writeStream'
# Almacena los resultados en la tabla temporal 'results_update_topic' permitiendo mayor versatilidad y control
# Para ver los resultados, esperar a que estén almacenados y ejecutar la consulta SQL

from pyspark.sql.functions import col, window
(
dataset
  .groupBy(window(col("timestamp"), "30 seconds", "5 seconds"), col("results"))
  .count()
  .writeStream \
  .outputMode("update") \
  .format("memory") \
  .option("truncate", "false") \
  .queryName("results_update_topic") \
  .start()
)


In [0]:
%sql
-- Mostrar los resultados del dataset
select * from results_update_topic

# Ejercicio 10
1. **Actualiza la columna `Results` de la tabla delta de food inspections creada en el ejercicio 1 al valor `No result`**
2. **Actualiza los datos de la tabla modificada en el punto 1 conforme vayan llegando elementos en Kafka**

---

Se aconseja parar todos los streams anteriores ya que el de este ejercicio suele hacer un uso intensivo de los recursos

In [0]:
# Paso 1.1

# Cargar la tabla Delta de food inspections
df_food_inspections = spark.read.format("delta").load("dbfs:/delta/food_inspections")

# Mostrar tabla
df_food_inspections.display()

In [0]:
# Paso  1.2 (continuación):

from pyspark.sql.functions import lit

# Actualizar la columna Results con el valor 'No result'
df_updated_results = df_food_inspections.withColumn("results", lit("No result"))

# Sobrescribir la tabla Delta con los nuevos valores
df_updated_results.write.format("delta").mode("overwrite").save("dbfs:/delta/food_inspections")

# Mostrar registros después de la actualización
df_updated_results.display()

In [0]:
# Paso 2.1 - Actualiza los datos de la tabla modificada en el punto 1 conforme vayan llegando elementos en Kafka

# Configuración servidor y topic de Kafka
kafka_server = "35.237.99.179:9094"
kafka_topic = "inspections"

In [0]:
# Paso 2.2

from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

# Definir el esquema para los datos de las inspecciones
schema = StructType([
    StructField("inspection_id", StringType(), True),
    StructField("dba_name", StringType(), True),
    StructField("aka_name", StringType(), True),
    StructField("license", StringType(), True),
    StructField("facility_type", StringType(), True),
    StructField("risk", StringType(), True),
    StructField("address", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("zip", StringType(), True),
    StructField("inspection_date", StringType(), True),
    StructField("inspection_type", StringType(), True),
    StructField("results", StringType(), True),
    StructField("violations", StringType(), True),
    StructField("latitude", StringType(), True),
    StructField("longitude", StringType(), True),
    StructField("location", StringType(), True)
])

# Leer el stream de Kafka
df_kafka = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("subscribe", kafka_topic) \
    .load()

# Convertir los datos de Kafka (json) al esquema estructurado
df_stream = df_kafka.selectExpr("CAST(value AS STRING) as json_string") \
    .withColumn("data", from_json(col("json_string"), schema)) \
    .select("data.*")



In [0]:
# Paso 2.3

from delta.tables import DeltaTable

# Función para actualizar la tabla Delta
def upsert_to_delta(microBatchOutputDF, batchId):
    # Mostrar el número de registros procesados en el batch actual
    print(f"Processing batch {batchId} with {microBatchOutputDF.count()} records")
    
    # Mostrar algunos registros del batch actual para debugging
    microBatchOutputDF.show(truncate=False)
    
    # Cargar la tabla Delta existente
    delta_table = DeltaTable.forPath(spark, "dbfs:/delta/food_inspections")
    
    # Realizar la operación de merge en la tabla Delta
    delta_table.alias("target").merge(
        microBatchOutputDF.alias("source"),
        "target.inspection_id = source.inspection_id"
    ).whenMatchedUpdate(set={
        "dba_name": col("source.dba_name"),
        "aka_name": col("source.aka_name"),
        "license": col("source.license"),
        "facility_type": col("source.facility_type"),
        "risk": col("source.risk"),
        "address": col("source.address"),
        "city": col("source.city"),
        "state": col("source.state"),
        "zip": col("source.zip"),
        "inspection_date": col("source.inspection_date"),
        "inspection_type": col("source.inspection_type"),
        "results": col("source.results"),
        "violations": col("source.violations"),
        "latitude": col("source.latitude"),
        "longitude": col("source.longitude"),
        "location": col("source.location")
    }).whenNotMatchedInsertAll().execute()

In [0]:
# Paso 2.4

from time import sleep

# Configurar el stream con un trigger para procesar los datos disponibles
query = df_stream.writeStream \
    .foreachBatch(upsert_to_delta) \
    .outputMode("update") \
    .option("checkpointLocation", "dbfs:/delta/checkpoints/food_inspections") \
    .trigger(once=True) \
    .start()

# Verificar los datos en la tabla Delta
spark.sql("SELECT * FROM delta.`dbfs:/delta/food_inspections`").show()

# Ejercicio 11: Diseño de Arquitectura de Datos

**Objetivo:** Diseñar una arquitectura de datos para manejar el sistema de inspecciones de sanidad descrito en los datasets utilizados en este proyecto.

**Descripción:**
Queremos que diseñen una arquitectura de datos eficiente y escalable para el sistema de inspecciones de sanidad. Los elementos a tener en cuenta:
* Para obtener los datos (CSVs incluidos en este proyecto) será necesario contectarse a un API REST
* Necesitamos limpiar los datos ya que en muchas veces vienen incompletos o con campos que no corresponden con la realidad
* Tenemos que cotejar las empresas con las empresas dadas de alta en el sistema, las cuales están almacenadas en un directorio en dbfs, en formato delta lake.
* En paralelo debemos diseñar un sistema de registro para los inspectores, donde podrán administrar y ver las inspecciones realizadas
* Que calcular algunas métricas a nivel de inspección tales como: Empresa con más inspecciones fallidas, o porcentaje de empresas con un mínimo de inspecciones. Estas métricas deberán actualizarse con un máximo de latencia de 30 minutos
* También definiremos métricas a nivel de inspectores, como inspectores con más inspecciones, ... Estás métricas sólo serán visibles a ciertos usuarios de la administración.
* Algunas de las métricas de inspecciones serán expuestas en un dashboard accesible por ciertos cargos del gobierno.
* Además debemos exponer los datos de inspecciones (con ciertas restricciones de visibilidad) a todos los ciudadanos a través de un API.

Este diseño debe abordar los siguientes puntos:

1. **Dominio de Datos:** Identifica y define dominios de datos relevantes para el sistema de inspecciones de sanidad.
2. **Propietarios del Dominio:** Asigna roles y responsabilidades para cada dominio de datos. ¿Quiénes serán los propietarios de estos datos?
4. **Infraestructura Autónoma:** Proporciona una visión general de cómo cada dominio manejará su infraestructura para la ingestión, almacenamiento, procesamiento y exposición de datos.
5. **Interoperabilidad:** Define cómo los distintos dominios se comunicarán y compartirán datos entre sí. ¿Qué estándares y protocolos se utilizarán?
6. **Gobernanza:** Establece principios de gobernanza de datos que aseguren la calidad, seguridad y cumplimiento de los datos en todos los dominios.
7. **Tecnologías Utilizadas:** Identifica las tecnologías y herramientas que emplearás para implementar tu diseño de arquitectura de datos (por ejemplo, Apache Spark, Kafka, Delta Lake, etc.).
8. **Evolución y Escalabilidad:** Proporciona una estrategia para evolucionar y escalar la arquitectura de datos conforme crezca la organización y la cantidad de datos.

**Entregable:**
Un documento en formato PDF que incluya un diagrama arquitectónico (usa la aplicación que consideres, por ejemplo, https://excalidraw.com/ ) y la descripción detallada de los puntos mencionados anteriormente. Además, justifica por qué elegiste este diseño y cómo crees que cumplirá con los requisitos del sistema de inspecciones de sanidad.
